In [ ]:
import torch
import numpy as np
import cv2
import os
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # For showing progress bar
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Dataset(object):
    
    
    def __getitem__(self, index):
        raise NotImplementedError
        
    def __len__(self):
        raise NotImplementedError
        
    def __add__(self, other):
        return ConcatDataset([self,other])

class MRI_Dataset(Dataset):
    def __init__(self, tumor_dir, healthy_dir, image_size=(128, 128)):
        self.tumor_images = []
        self.healthy_images = []
        self.labels = []
        
        self.image_size = image_size
        
        # Load tumor images
        for filename in os.listdir(tumor_dir):
            img = cv2.imread(os.path.join(tumor_dir, filename))
            img = cv2.resize(img, self.image_size)
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])  # Convert from BGR to RGB
            self.tumor_images.append(img)
            self.labels.append(1)  # Tumor = 1
        
        # Load healthy images
        for filename in os.listdir(healthy_dir):
            img = cv2.imread(os.path.join(healthy_dir, filename))
            img = cv2.resize(img, self.image_size)
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])  # Convert from BGR to RGB
            self.healthy_images.append(img)
            self.labels.append(0)  # Healthy = 0
        
        self.images = np.array(self.tumor_images + self.healthy_images)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0  # Normalize
        label = self.labels[idx]
        return {'image': torch.tensor(image).permute(2, 0, 1), 'label': torch.tensor(label, dtype=torch.float32)}


# Load dataset paths
train_tumor_dir = './Dataset/Training/aug_tumor'
train_healthy_dir = './Dataset/Training/aug_notumor'
test_tumor_dir = './Dataset/Testing/aug_tumor'  # Separate testing directory
test_healthy_dir = './Dataset/Testing/aug_notumor'  # Separate testing directory

# Create Dataset and DataLoader for training
train_dataset = MRI_Dataset(train_tumor_dir, train_healthy_dir)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create Dataset and DataLoader for testing
test_dataset = MRI_Dataset(test_tumor_dir, test_healthy_dir)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 1)  # Binary classification
        
        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(-1, 256 * 8 * 8)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for binary classification
        return x

# Define device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
model = CNN().to(device)

# Define Loss function and Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set number of epochs (adjust as needed)
epochs = 30

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()  # Zero the gradients before backprop
        
        # Forward pass
        outputs = model(images)  # Forward pass through the network
        
        # Compute the loss
        loss = criterion(outputs.squeeze(), labels)  # Squeeze the output to match the label shape
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Track the running loss
        running_loss += loss.item()
        
        # Calculate accuracy
        predicted = (outputs.squeeze() > 0.5).float()  # Predicted class: 1 if output > 0.5, else 0
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    # Average loss and accuracy for the epoch
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    # Validation loop (using the separate test dataset)
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation during validation
        for batch in test_loader:
            images = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(images)  # Forward pass through the model
            loss = criterion(outputs.squeeze(), labels)  # Compute validation loss
            val_loss += loss.item()
            
            # Calculate validation accuracy
            predicted = (outputs.squeeze() > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Average validation loss and accuracy
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

# Save model after all epochs
torch.save(model.state_dict(), 'final_model.pth')

